# Open converted files

## Open a converted netCDF or Zarr dataset

Converted netCDF files can be opened with the open_converted function that returns a lazy loaded EchoData object (only metadata are read during opening):

```python
import echopype as ep
file_path = "./converted_files/file.nc"      # path to a converted nc file
ed = ep.open_converted(file_path)            # create an EchoData object
```

Likewise, specify the path to open a Zarr dataset. To open such a dataset from cloud storage, use the same `storage_options` parameter as with [open_raw](convert.html#aws-s3-access). For example:

```python
s3_path = "s3://s3bucketname/directory_path/dataset.zarr"     # S3 dataset path
ed = ep.open_converted(s3_path, storage_options={"anon": True})
```

## Combine EchoData objects

Converted data found in multiple files corresponding to the same instrument deployment can be combined into a single `EchoData` object. First assemble a list of `EchoData` objects from the converted files (netCDF or Zarr). Then apply `combine_echodata` on this list to combine all the data into a single `EchoData` object in memory:

```python
ed_list = []
for converted_file in ["convertedfile1.nc", "convertedfile2.nc"]:
   ed_list.append(ep.open_converted(converted_file))

combined_ed = ep.combine_echodata(ed_list)
```

## EchoData object

`EchoData` is an object that conveniently handles raw converted data from either raw instrument files (via `open_raw`) or previously converted and standardized raw files (via `open_converted`). It is essentially a container for multiple `xarray Dataset` objects, where each such object corresponds to one of the netCDF4 groups specified in the SONAR-netCDF4 convention followed by echopype. `EchoData` objects are used for conveniently accessing and exploring the echosounder data, for calibration and other processing, and for [serializing into netCDF4 or Zarr file formats](convert.html#file-export).

A sample `EchoData` object is presented below using the `Dataset` HTML browser generated by `xarray`, collected into SONAR-netCDF4 groups. Select each group and drill down to variables and attributes to examine the structure and representative content of an `EchoData` object.

In [1]:
from pathlib import Path
import echopype as ep
bucket = "ncei-wcsd-archive"
rawdirpath = "data/raw/Bell_M._Shimada/SH1707/EK60/Summer2017-D20170728-T181619.raw"
s3raw_fpath = f"s3://{bucket}/{rawdirpath}"
ed = ep.open_raw(s3raw_fpath, sonar_model='EK60', storage_options={'anon': True})
# Manually populate additional metadata about the dataset and the platform
# -- SONAR-netCDF4 Top-level Group attributes
ed['Top-level'].attrs['title'] = "2017 Pacific Hake Acoustic Trawl Survey"
ed['Top-level'].attrs['summary'] = (
    f"EK60 raw file {s3raw_fpath} from the {ed['Top-level'].attrs['title']}, "
    "converted to a SONAR-netCDF4 file using echopype."
)
# -- SONAR-netCDF4 Platform Group attributes
ed['Platform'].attrs['platform_type'] = "Research vessel"
ed['Platform'].attrs['platform_name'] = "Bell M. Shimada"
ed['Platform'].attrs['platform_code_ICES'] = "315"

13:08:29  parsing file Summer2017-D20170728-T181619.raw, time of first ping: 2017-Jul-28 18:16:19


In [2]:
ed

<EchoData: standardized raw data from Internal Memory>
Top-level: contains metadata about the SONAR-netCDF4 file format.
├── Environment: contains information relevant to acoustic propagation through water.
├── Platform: contains information about the platform on which the sonar is installed.
│   └── NMEA: contains information specific to the NMEA protocol.
├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
├── Sonar: contains specific metadata for the sonar system.
│   └── Beam_group1: contains backscatter data (either complex samples or uncalibrated power samples) and other beam or channel-specific data, including split-beam angle data when they exist.
└── Vendor specific: contains vendor-specific information about the sonar and the data.